In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.nn import functional as F
from tqdm import trange

In [ ]:
batch_size = 512 # B
block_size = 512 # T
# C = 65 -> vocab length
head_size = 16
num_steps = 5000
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
learning_rate = 1e-3
n_embd = 64
n_head = 4
n_layer = 4
dropout_percent = 0.0
eval_interval = 1000
eval_iters = 10

In [ ]:
from datasets import load_dataset
ds = load_dataset("Trelis/tiny-shakespeare")

In [ ]:
ds_ = ds['train']['Text']

In [ ]:
chars = sorted(set(''.join(ds_))) # vocabulory
vocab_size = len(chars)
len(chars)

In [ ]:
encoder_dict = {k:v for k,v in zip(chars, range(len(chars)))}
decoder_dict = {v:k for k,v in zip(chars, range(len(chars)))}

# Encoder, Decoder
encode = lambda x: [encoder_dict[letter] for letter in x]
decode = lambda x: ''.join([decoder_dict[letter] for letter in x])
encode('hello'), decode([46, 43, 50, 50, 53])

In [ ]:
ds_all = '\n'.join(ds_)
ds_encoded = encode(ds_all)

In [ ]:
n = int(.9*len(ds_encoded))
train_data = ds_encoded[:n]
val_data = ds_encoded[n:]

In [ ]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    data = torch.tensor(data)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [ ]:
# def get_batch(data, block_size=block_size, batch_size=batch_size):
#     # print(len(data))
#     ix = torch.randint(len(data)-block_size, (batch_size,)) # these numbers are the start of each batch
#     xy = torch.tensor([[data[i] for i in range(ix[i],ix[i]+block_size+1)] for i in range(batch_size)])
#     xb = xy[:, :block_size]
#     yb = xy[:,1:]
#     # print(xb.shape, yb.shape) # (B x T)
#     return xb, yb

# get_batch(train_data)

In [ ]:
# write code to estimate loss with train and test dataset

In [ ]:
class Head(nn.Module):
    '''one head in self attention'''
    def __init__(self):
        super().__init__()
        self.key = nn.Linear(n_embd, int(n_embd/n_head), bias=False)
        self.query = nn.Linear(n_embd, int(n_embd/n_head), bias=False)
        self.value = nn.Linear(n_embd, int(n_embd/n_head), bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) # registers are saved along with the model, but are never updated during training. They dont count as model parameters
        self.dropout = nn.Dropout(dropout_percent)
    
    def forward(self, x):
        B ,T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)

        # compute attention score
        wei = q @ k.transpose(-2, -1) / C**0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # mask future tokens
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        
        return wei@v

In [ ]:
class MultiHeadAttention(nn.Module):
    '''multiple attention heads in parallel'''

    def __init__(self, num_heads):
        super().__init__()
        self.heads = nn.ModuleList([Head() for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd) # to apply a linear transformation to concat heads
        self.dropout = nn.Dropout(dropout_percent)
    
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [ ]:
class FeedForward(nn.Module):
    
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout_percent),
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.multihead = MultiHeadAttention(n_head)
        self.ff = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd) 
        self.ln2 = nn.LayerNorm(n_embd) 
    
    def forward(self, x):
        x = x + self.multihead(self.ln1(x)) # pre-normalisation and residual connections
        x = x + self.multihead(self.ln2(x)) 
        return x
        

In [ ]:
class LLM(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[TransformerBlock(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None: # during inference
            loss = None
        else:
            # during training
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss 
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond) # loss wont exist during inferencec
            logits = logits[:, -1, :] # becomes (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [ ]:

model = LLM()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:

for iter in trange(num_steps):

    if iter % eval_interval == 0 or iter == num_steps - 1:
        losses = estimate_loss()
        print(f"Step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

: 